In [1]:
import os
import json
import numpy as np
import pandas as pd

class Experiment1:
    def __init__(self, run_id=0, experiment_id=0, autosave=True, autosave_dir="."):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._id_names = ["run_id", "experiment_id"]
        self.table = pd.DataFrame(columns=self._id_names)
        
        self._static_params = None
                
        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            self.benchmark_number = len([d for d in os.listdir(self.autosave_dir) if "benchmark" in d])
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_number}/tables"
        else:
            self.benchmark_number = None
            self.table_dir = None
    
    def __enter__(self, *args, **kwargs):
        self.start(args, kwargs)
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.end()
        
    def __next__(self):
        self.run_id += 1
        return self.run_id

    # 前処理
    def start(self, *args, **kwargs):
        self._static_params = kwargs

    #　後処理
    def end(self):
        cols = ["run_id", "experiment_id"]
        self.table[cols] = self.table[cols].astype(int)
        
    def insert_into_table(self, obj):
        self.table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in self._static_params.items():
            self.table.loc[self.run_id, k] = v
        self.table.loc[self.run_id, "results"] = json.dumps(obj)
        next(self)
                
    def load(self, load_file=None):
        if self.autosave:
            self.table = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self.table = pd.read_csv(load_file)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.table.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.table.to_csv(save_file)
        

class Experiment2(pd.DataFrame):
    # temporary properties
    _internal_names = pd.DataFrame._internal_names + ["autosave", "autosave_dir", "id_names", "run_id", "experiment_id"]
    _internal_names_set = set(_internal_names)

    # normal properties
    _metadata = ["added_property"]

    @property
    def _constructor(self):
        return Experiment2

    def __init__(
        self,
        data=None,
        index=None,
        columns=None,
        dtype=None,
        copy=None,
        size=30,
        run_id=0,
        experiment_id=0,
        autosave=True,
        autosave_dir=".",
    ):
        data = [[np.nan, np.nan]] * size
        data = None
        self.id_names = ["run_id", "experiment_id"]
        self.id_names = None
        self.run_id = run_id
        self.experiment_id = experiment_id
        super().__init__(data=data, index=None, columns=self.id_names, dtype=None, copy=None)
        # self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
        self.autosave = autosave
        self.autosave_dir = autosave_dir
    
    def start(self):
        pass
    
    def end(self):
        pass
        
    def next(self):
        self.run_id += 1
        self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
    
    def insert_dict(self, obj):
        for k, v in obj.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            self.loc[self.run_id, k] = v

    def load(self, filepath=None):
        if self.autosave:
            self = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self = pd.read_csv(filepath)
        return self

    def save(self, filepath=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.to_csv(filepath)


class Experiment3:
    def __init__(self, run_id=0, experiment_id=0, autosave=True, autosave_dir="."):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._id_names = ["run_id", "experiment_id"]
        self._table = None
        self._table_dtypes = {"run_id": int, "experiment_id": int}
            
        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            self.benchmark_id = len([d for d in os.listdir(self.autosave_dir) if "benchmark" in d])
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_id}/tables"
        else:
            self.benchmark_id = None
            self.table_dir = None
            
    @property
    def table(self):
        return self._table
    
    def __enter__(self, *args, **kwargs):
        self._table = pd.DataFrame(columns=self._id_names)
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        for (k, true), act in zip(self._table_dtypes.items(), self.table.dtypes):
            if true != act:
                self.table[k] = self.table[k].astype(true)
        
    def __next__(self):
        self.run_id += 1
        return self.run_id        
        
    def insert_into_table(self, obj):
        self.table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in obj.items():
            if isinstance(v, dict):
                 v = json.dumps(v)
            
            self.table.loc[self.run_id, k] = v
            self._table_dtypes[k] = type(v)
        next(self)
                
    def load(self, load_file=None):
        if self.autosave:
            self.table = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self.table = pd.read_csv(load_file)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.table.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.table.to_csv(save_file)


In [2]:
# Responseのサンプル
response_sample = {
    "type": "Response",
    "version": {"sampleset_schema": "3.1.0"},
    "num_variables": 20,
    "num_rows": 5,
    "sample_data": {
        "type": "array",
        "data": [[123424], [747616], [256132], [358510], [598528]],
        "data_type": "uint32",
        "shape": [5, 1],
        "use_bytes": False,
    },
    "sample_type": "int8",
    "sample_packed": True,
    "vectors": {
        "energy": {
            "type": "array",
            "data": [-207.0, -243.0, -272.0, -264.0, -157.0],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "num_occurrences": {
            "type": "array",
            "data": [1, 1, 1, 1, 1],
            "data_type": "int64",
            "shape": [5],
            "use_bytes": False,
        },
    },
    "variable_labels": [
        "_leint_knapsack_constraint[0]",
        "_leint_knapsack_constraint[1]",
        "_leint_knapsack_constraint[2]",
        "_leint_knapsack_constraint[3]",
        "_leint_knapsack_constraint[4]",
        "_leint_knapsack_constraint[5]",
        "_leint_knapsack_constraint[6]",
        "_leint_knapsack_constraint[7]",
        "_leint_knapsack_constraint[8]",
        "_leint_knapsack_constraint[9]",
        "x[0]",
        "x[1]",
        "x[2]",
        "x[3]",
        "x[4]",
        "x[5]",
        "x[6]",
        "x[7]",
        "x[8]",
        "x[9]",
    ],
    "variable_type": "BINARY",
    "info": {
        "system": [],
        "sampling_time": 4100.264981389046,
        "execution_time": 751.1120289564133,
        "list_exec_times": {
            "type": "array",
            "data": [
                858.2137525081635,
                771.5118117630482,
                698.3112543821335,
                719.3121127784252,
                708.211213350296,
            ],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "schedule": {
            "beta_max": 4.605170185988092,
            "beta_min": 4.652244285330389e-06,
            "num_sweeps": 1000,
        },
    },
}

In [166]:
# Experiment1を使ったサンプルコード
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment1()
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start(**args)
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            results = {"step": step,"response": response_sample}
            experiment.insert_into_table(results)
            
        experiment.end()


In [167]:
experiment.table

,run_id,experiment_id,num_sweeps,num_reads,results
0,0,0,10.0,1.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
1,1,0,10.0,1.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
2,2,0,10.0,1.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
3,3,0,10.0,2.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
4,4,0,10.0,2.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
5,5,0,10.0,2.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
6,6,0,10.0,3.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
7,7,0,10.0,3.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
8,8,0,10.0,3.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
9,9,0,20.0,1.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."


In [128]:
experiment.save()

In [105]:
# Experiment2を使ったサンプルコード
experiment = Experiment2()
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment2()
run_id = 0
experiment_id = 0
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start()
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            args["step"] = step
            args["response"] = response_sample
            experiment.insert_dict(args)
            experiment.next()
        experiment.end()



In [106]:
experiment

,run_id,experiment_id,num_sweeps,num_reads,step,response
0,0.0,0.0,10.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
1,1.0,0.0,10.0,1.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
2,2.0,0.0,10.0,1.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
3,3.0,0.0,10.0,2.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
4,4.0,0.0,10.0,2.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
5,5.0,0.0,10.0,2.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
6,6.0,0.0,10.0,3.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
7,7.0,0.0,10.0,3.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
8,8.0,0.0,10.0,3.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
9,9.0,0.0,20.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."


In [225]:
# Experiment3を使ったサンプルコード
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

with Experiment3() as experiment:
    for num_sweeps in static_params["num_sweeps"]:
        for num_reads in static_params["num_reads"]:
            for step in steps:
                # ここではsolve関数は上のresponse_sampleを返す想定
                # response_sample = solve()
                
                # experiment.tableに登録するrecordを辞書型で作成
                record = {
                    "step": step,
                    "num_sweeps": num_sweeps,
                    "num_reads": num_reads,
                    "results": response_sample
                }
                experiment.insert_into_table(record)

In [226]:
experiment

In [109]:
df = pd.DataFrame()
df.loc["a", 0] = 1
df.loc["b", 1] = 1
df.loc["c", [0, 1]] = [1, 1]

df.dropna()

,0,1
c,1.0,1.0


In [116]:
df = Experiment2()
df.loc["a", 0] = 1
df.loc["b", 1] = 1
df.loc["c", [0, 1]] = [1, 1]

ValueError: cannot set a frame with no defined index and a scalar

In [3]:
import os
import json
from typing import Union
import pandas as pd


class Experiment:
    def __init__(
        self,
        run_id: int = 0,
        experiment_id: Union[int, str] = 0,
        benchmark_id: Union[int, str] = None,
        autosave: bool = True,
        autosave_dir: str = ".",
    ):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir

        self._id_names = ["run_id", "experiment_id"]
        self._table = pd.DataFrame(columns=self._id_names)
        self._table_dtypes = {"run_id": int, "experiment_id": type(self.experiment_id)}

        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            if benchmark_id is None:
                benchmark_id = sum(
                    [
                        os.path.isdir(f"{self.autosave_dir}/{d}")
                        for d in os.listdir(self.autosave_dir)
                        if "benchmark" in d
                    ]
                )
            self.benchmark_id = benchmark_id
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_id}/tables"
        else:
            self.benchmark_id = None
            self.table_dir = None

    @property
    def table(self):
        return self._table

    def __enter__(self, *args, **kwargs):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        pass

    def __next__(self):
        self.run_id += 1
        return self.run_id

    def insert_into_table(self, record, replace=True):
        self._table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in record.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            if isinstance(v, list):
                v = str(v)

            self._table.loc[self.run_id, k] = v
            self._table[k] = self._table[k].astype(type(v))
            self._table_dtypes[k] = type(v)
        if replace:
            next(self)

    def load(self, load_file=None):
        if self.autosave:
            self._table = pd.read_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv", index_col=0
            )
        else:
            self._table = pd.read_csv(load_file, index_col=0)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self._table.to_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv"
            )
        else:
            self._table.to_csv(save_file)

In [25]:
experiment_id="sample"
benchmark_id="test"
experiment = Experiment(experiment_id=experiment_id, benchmark_id=benchmark_id)
experiment.table

,run_id,experiment_id


In [26]:
# Example 1
# 最も単純な使い方
# ユーザ定義のsolverの帰り値（何でも良い）
sample_response = {"hoge": {"fuga": 1}}
for param in [10, 100, 1000]:
    for step in range(3):
        # solverは上のsample_responseを返す想定
        # sample_response = solver()
        # experiment.tableに登録するrecordを辞書型で作成
        record = {
            "step": step,
            "param": param,
            "results": sample_response,
        }
        experiment.insert_into_table(record)
experiment.save()

In [145]:
import jijzept as jz
import jijmodeling as jm

d = jm.Placeholder("d", dim=1)
n = d.shape[0]

x = jm.Binary("x", shape=(n, ))

problem = jm.Problem("sample")

i = jm.Element("i", (0, n))

problem += jm.Sum(i, d[i]*x[i])
problem += jm.Constraint("onehot", x[:] == 1)
problem += jm.Constraint("onehot2", x[:] == 1)

data = {"d": [1, 2, -1, -3, -1]}

sampler = jz.JijSASampler(config="/home/azureuser/.config/jijzept/config.toml")
response = sampler.sample_model(problem, data, {'onehot': 4},  num_reads=5)


uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is d62dffd0616249dab9d702494579612c.


In [146]:
decoded = problem.decode(response, data, {})

In [3]:
print(decoded.__class__.__name__)

DecodedSamples


In [3]:
import openjij as oj

a = oj.Response

sampler = oj.SASampler()
pyq_obj = problem.to_pyqubo(data, {})
bqm = pyq_obj.compile().to_bqm(feed_dict={"onehot": 4})
sampler = oj.SASampler(num_reads=5)
response = sampler.sample(bqm)


In [328]:
import dimod

isinstance(response, dimod.SampleSet)
# print(response.energies)
print(response.record.energy)
print(response.record.num_occurrences)

[-3. -3. -3. -3. -3.]
[-3. -3. -3. -3. -3.]
[1 1 1 1 1]


In [331]:
x = ["a", "b", "c"]
x.remove("a")

x

None


['b', 'c']

In [61]:
import numpy
import pandas as pd
from typing import List

x = [[np.zeros((2, 2))]]

df = pd.DataFrame(x, columns=["a"])
df.astype({"a": object})

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'a' not found in columns."

In [76]:
_COLUMN_DTYPE = {
    "energies": object,
    "energy_min": float,
    "energy_mean": float,
    "energy_std": float,
    "objectives": object,
    "obj_min": float,
    "obj_mean": float,
    "obj_std": float,
    "num_feasible": int,
    "num_samples": int,
    "{const_name}_violations": object,
    "{const_name}_violation_min": float,
    "{const_name}_violation_mean": float,
    "{const_name}_violation_std": float
}

df = pd.DataFrame(columns=_TABLE_DTYPE.keys())
df.rename({df.columns[-1]: df.columns[-1].format(const_name="onehot")}, axis=1, inplace=True)
df

,energies,energy_min,energy_mean,energy_std,objectives,obj_min,obj_mean,obj_std,num_feasible,num_samples,onehot_violations


In [75]:
_TABLE_DTYPE

{'energies': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objectives': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object}

list

In [188]:
import os
import json
import uuid
import dimod
import numpy as np
import pandas as pd
from typing import Union

_ID_COLUMN_DTYPES = {
    "run_id": object,
    "experiment_id": object,
    "benchmark_id": object,
}

_FIXED_COLUMN_DTYPES = {
    
    "energy": object,
    "energy_min": float,
    "energy_mean": float,
    "energy_std": float,
    "objective": object,
    "obj_min": float,
    "obj_mean": float,
    "obj_std": float,
    "num_feasible": int,
    "num_samples": int,
}

_VIOLATION_COLUMN_DTYPES = {
    "{const_name}_violations": object,
    "{const_name}_violation_min": float,
    "{const_name}_violation_mean": float,
    "{const_name}_violation_std": float,
}


class Experiment:
    def __init__(
        self,
        experiment_id: Union[int, str] = None,
        benchmark_id: Union[int, str] = None,
        autosave: bool = True,
        autosave_dir: str = ".",
    ):        
        self._table = _Table(experiment_id=experiment_id, benchmark_id=benchmark_id)
        self._artifact = {}
        
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._dirs = _Dir(autosave=autosave, autosave_dir=autosave_dir)
        
    @property
    def run_id(self):
        return self._table.run_id
    
    @property
    def experiment_id(self):
        return self._table.experiment_id
    
    @property
    def benchmark_id(self):
        return self._table.benchmark_id
    
    @property
    def table(self):
        return self._table.data
    
    @property
    def artifact(self):
        return self._artifact

    def __enter__(self):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        pass
    
    def __next__(self):
        self._table.current_index += 1
        self._table.run_id = uuid.uuid4()
        return self.run_id

    def start(self):
        self._table.current_index = 0
        self._table.run_id = uuid.uuid4()
        self._table.experiment_id = uuid.uuid4() if self.experiment_id is None else self.experiment_id
        self._table.benchmark_id = uuid.uuid4() if self.benchmark_id is None else self.benchmark_id
        self._dirs.make_dirs(benchmark_id=self.benchmark_id)
        return self

    def end(self):
        pass

    def insert_into_table(self, record, replace=True):
        def _reconstruct_record():
            _new_record = {}
            for _k, _v in record.items():
                if isinstance(_v, dimod.SampleSet):                                        
                    _energies = _v.record.energy
                    _num_occurrences = response.record.num_occurrences
                    
                    _columns = experiment._table.get_energy_columns() + experiment._table.get_num_columns()
                    _values = [
                        list(_energies),
                        _energies.min(),
                        _energies.mean(),
                        _energies.std(),
                        list(_num_occurrences),
                        np.nan,
                        np.nan
                        ]
                    for _new_k, _new_v in zip(_columns, _values):
                        _new_record[_new_k] = _new_v      
                elif _v.__class__.__name__ == "DecodedSamples":
                    _energies = _v.energies
                    _objectives = _v.objectives
                    
                    _constraint_violations = {}
                    for _violation in _v.constraint_violations:
                        for _const_name, _value in _violation.items():
                            if _const_name in _constraint_violations.keys():
                                _constraint_violations[_const_name].append(_value)
                            else:
                                _constraint_violations[_const_name] = [_value]
                    
                    _columns = experiment._table.get_energy_columns()
                    _columns += experiment._table.get_objective_columns()
                    _columns += experiment._table.get_num_columns()
                    
                    _values = [
                        list(_energies),
                        _energies.min(),
                        _energies.mean(),
                        _energies.std(),
                        list(_objectives),
                        _objectives.min(),
                        _objectives.mean(),
                        _objectives.std(),
                        np.nan,
                        len(_v.feasibles()),
                        len(_v.data)
                    ]
                    for _const_name, _violation_values in _constraint_violations.items():
                        _violation_values = np.array(_violation_values)
                        _columns += self._table.rename_violation_columns(_const_name)
                        _values += [
                            list(_violation_values),
                            _violation_values.min(),
                            _violation_values.mean(),
                            _violation_values.std()
                            ]

                    for _new_k, _new_v in zip(_columns, _values):
                        _new_record[_new_k] = _new_v
                else:
                    _new_record[_k] = _v
            return _new_record
                
        index = self._table.current_index
        ids = self._table.get_id_columns()
        self._table.data.loc[index, ids] = [self.run_id, self.experiment_id, self.benchmark_id]
        
        record = _reconstruct_record()
        for k, v in record.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            elif isinstance(v, list):
                v = str(v)

            self._table.data.loc[index, k] = v
            self._table.data[k] = self._table.data[k].astype(type(v))
    
        if replace:
            next(self)
            
    def update_artifact(self, results):
        self._artifact.update(results)

    def load(self, load_file=None):
        if self._autosave:
            self._table = pd.read_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv", index_col=0
            )
        else:
            self._table = pd.read_csv(load_file, index_col=0)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self._table.to_csv(
                f"{self._table_dir}/experiment_id_{self.experiment_id}.csv"
            )
        else:
            self._table.to_csv(save_file)


class _Table:
    id_dtypes = {
        "run_id": object,
        "experiment_id": object,
        "benchmark_id": object,
    }
    energy_dtypes = {
        "energy": object,
        "energy_min": float,
        "energy_mean": float,
        "energy_std": float,
    }

    objective_dtypes = {
        "objective": object,
        "obj_min": float,
        "obj_mean": float,
        "obj_std": float,
    }

    num_dtypes = {
        "num_occurances": object,
        "num_feasible": int,
        "num_samples": int,
    }

    violation_dtypes = {
        "{const_name}_violations": object,
        "{const_name}_violation_min": float,
        "{const_name}_violation_mean": float,
        "{const_name}_violation_std": float,
    }
    
    __dtypes_names = [
        "id_dtypes",
        "energy_dtypes",
        "objective_dtypes",
        "num_dtypes",
        "violation_dtypes"
        ]
    
    def __init__(self, run_id=None, experiment_id=None, benchmark_id=None):
        columns = self.get_columns()
        self._data = pd.DataFrame(columns=columns)
        self._current_index = 0
        
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.benchmark_id =  benchmark_id

    @property
    def data(self):
        return self._data
    
    @property
    def current_index(self):
        return self._current_index
    
    @current_index.setter
    def current_index(self, index):
        self._current_index = index
    
    def get_columns(self):
        c = []
        for name in self.__dtypes_names:
            c += list(getattr(self, name).keys())
        return c
    
    def get_id_columns(self):
        return list(self.id_dtypes.keys())
    
    def get_energy_columns(self):
        return list(self.energy_dtypes.keys())
    
    def get_objective_columns(self):
        return list(self.objective_dtypes.keys())
    
    def get_num_columns(self):
        return list(self.num_dtypes.keys())
    
    def get_violation_columns(self):
        return list(self.violation_dtypes.keys())
    
    def rename_violation_columns(self, const_name):
        columns = self.get_violation_columns()
        for i, c in enumerate(columns):
            columns[i] = c.format(const_name=const_name)
        return columns
    
    def get_dtypes(self):
        t = {}
        for name in self.__dtypes_names:
            t |= getattr(self, name)
        return t
    

class _Dir:
    def __init__(self, autosave, autosave_dir):
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._table_dir = None
        self._artifact_dir = None
        
    @property
    def table_dir(self):
        return self._table_dir
    
    @property
    def artifact_dir(self):
        return self._artifact_dir
        
    def make_dirs(self, benchmark_id):
        if self.autosave:
            os.makedirs(self.autosave_dir, exist_ok=True)
            self._table_dir = f"{self.autosave_dir}/benchmark_{benchmark_id}/tables"
            self._artifact_dir = f"{self.autosave_dir}/benchmark_{benchmark_id}/artifacts"
    
            
def _insert_dimod_sampleset(experiment, response):
    index = 0
    energies = response.record.energy
    energy_min = energies.min()
    energy_mean = energies.mean()
    energy_std = energies.std()
    num_occurrences = response.record.num_occurrences
    
    columns = experiment._table.get_energy_columns() + experiment._table.get_num_columns()
    values = [
        energies,
        energy_min,
        energy_mean,
        energy_std,
        num_occurrences,
        np.nan,
        np.nan
        ]
    for c, v in zip(columns, values):
        print(index, c, v)
        experiment._table.data.loc[index, c] = json.dumps(list(v))
    
    return experiment
    

In [61]:
energies = response.record.energy

experiment = Experiment(benchmark_id="test")
experiment.start()
experiment._table.data.loc[0, "run_id"] = uuid.uuid4()
experiment._table.data.loc[0, "experiment_id"] = uuid.uuid4()
experiment._table.data.loc[0, "benchmark_id"] = "test"
experiment = _insert_dimod_sampleset(experiment, response)
experiment.table

0 energy [-3. -3. -3. -3. -3.]
0 energy_min -3.0
0 energy_mean -3.0
0 energy_std 0.0
0 num_occurances [1 1 1 1 1]
0 num_feasible nan
0 num_samples nan


,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std
0,a180c531-805d-4ce9-ad7f-2c7cda78a994,10bc44ff-3ab5-4c96-891d-7c123fc4a367,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
sample_response = {"hoge": {"fuga": 1}}

experiment = Experiment(benchmark_id="test")

with experiment.start() as e:
    for param in [10, 100, 1000]:
        for step in range(3):
            # solverは上のsample_responseを返す想定
            # sample_response = solver()
            # experiment.tableに登録するrecordを辞書型で作成
            record = {
                "step": step,
                "param": param,
                # "results": response,
                "results": decoded,
            }
            experiment.insert_into_table(record)

In [190]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,...,step,param,onehot_violations,onehot_violation_min,onehot_violation_mean,onehot_violation_std,onehot2_violations,onehot2_violation_min,onehot2_violation_mean,onehot2_violation_std
0,05966a90-f271-4fd3-8300-f83eb8fa18b8,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
1,5b0bc8d0-4a84-4fcb-bdd1-fca260718699,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
2,3bde49d4-2c4f-4ad4-b9ef-5097b8814a01,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
3,86326e1a-d8a3-4621-b866-5e93b66f7d7e,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
4,16068876-a837-488c-a5eb-5923ecf5647b,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
5,766fbd23-6cb1-49f4-a86d-173973a47e7b,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
6,be87aafc-ff41-41ac-ad73-baca0f694e32,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
7,1d865ff2-dcf4-4cfe-9b2f-526cf0c57db7,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
8,625fd3d2-b39d-4634-846e-1844a578389c,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0


In [147]:
decoded.constraint_violations

array([{'onehot': 0.0, 'onehot2': 0.0}, {'onehot': 0.0, 'onehot2': 0.0},
       {'onehot': 0.0, 'onehot2': 0.0}, {'onehot': 0.0, 'onehot2': 0.0},
       {'onehot': 0.0, 'onehot2': 0.0}], dtype=object)

In [149]:
decoded.constraint_expr_value

array([{'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}}], dtype=object)

In [116]:
for violations in decoded.constraint_violations:
    print(violations)

{'onehot': 0.0}


In [118]:
x = {"x": 1}
x

{'x': 1}

In [114]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std,step,param
0,257f1517-d01a-4e4d-b908-3d99f4b4cdeb,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,10
1,2295f933-6535-49ad-8c45-0e485766cb7c,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,10
2,63f333c6-2093-4094-9ea8-bffaeb29143a,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,10
3,9b69d63a-c9f3-4a45-98ee-2c856681ef4b,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,100
4,a0d3dc90-9a76-4383-8818-d2183ab31bad,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,100
5,ba13a8d7-5e3e-4237-b00a-9c58ed482772,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,100
6,2a55f1a3-82d8-41ec-a4a4-a4e55e81f093,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,1000
7,ee2de150-1bfc-48da-913f-dfdcbf2fd1df,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,1000
8,2f3b01db-1a5a-4cb0-89f7-30bdc44edf0c,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,1000


In [95]:
x = {"a":1 , "b": 2, "c": 3}

for k in ["a", "b"]:
    del x[k]
x

{'c': 3}

In [51]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std,step,param
0,ce6fe767-b049-4384-b954-010cfe4e6850,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,10
1,02f8c0a7-8c58-47f3-be1b-fba0e174238c,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,10
2,8f52ea2f-7b4e-4c3a-886f-22f14448070d,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,10
3,1d907818-c21a-4736-8f98-04b52c5bbe56,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,100
4,acc14cec-2f94-423a-9a77-0b379f59d2ca,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,100
5,7d21341a-f047-40f2-a6bd-7b6c4607f667,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,100
6,92e0206f-6bc1-4e0e-8c04-ffded51d2cdf,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,1000
7,c5153b07-6d66-45e6-b8e8-de1df8728109,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,1000
8,a8d3ad3d-a639-44ec-b6a6-75600bba0463,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,1000


In [239]:
import os
import json
import uuid
import pandas as pd
from typing import Dict, List, Union
from dataclasses import dataclass, field


class _TableColumn:
    id_dtypes = {
        "run_id": object,
        "experiment_id": object,
        "benchmark_id": object,
    }
    energy_dtypes = {
        "energy": object,
        "energy_min": float,
        "energy_mean": float,
        "energy_std": float,
    }

    objective_dtypes = {
        "objective": object,
        "obj_min": float,
        "obj_mean": float,
        "obj_std": float,
    }

    num_dtypes = {
        "num_feasible": int,
        "num_samples": int,
    }

    violation_dtypes = {
        "{const_name}_violations": object,
        "{const_name}_violation_min": float,
        "{const_name}_violation_mean": float,
        "{const_name}_violation_std": float,
    }
    
    __dtypes_names = [
        "id_dtypes",
        "energy_dtypes",
        "objective_dtypes",
        "num_dtypes",
        "violation_dtypes"
        ]
    
    def get_dtypes(self):
        t = {}
        for name in self.__dtypes_names:
            t |= getattr(self, name)
        return t
    
    
    def get_columns(self):
        c = []
        for name in self.__dtypes_names:
            c += list(getattr(self, name).keys())
        return c
    
    
    



['run_id', 'experiment_id', 'benchmark_id', 'energy', 'energy_min', 'energy_mean', 'energy_std', 'objective', 'obj_min', 'obj_mean', 'obj_std', 'num_feasible', 'num_samples', '{const_name}_violations', '{const_name}_violation_min', '{const_name}_violation_mean', '{const_name}_violation_std']


In [115]:
df = pd.DataFrame(columns=_get_columns()).astype(_get_column_dtypes())
df.dtypes

run_id                          object
experiment_id                   object
benchmark_id                    object
energy                          object
energy_min                     float64
energy_mean                    float64
energy_std                     float64
objective                       object
obj_min                        float64
obj_mean                       float64
obj_std                        float64
num_feasible                     int64
num_samples                      int64
{const_name}_violations         object
{const_name}_violation_min     float64
{const_name}_violation_mean    float64
{const_name}_violation_std     float64
dtype: object

In [109]:
_get_column_dtypes()

{'run_id': object,
 'experiment_id': object,
 'benchmark_id': object,
 'energy': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objective': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object,
 '{const_name}_violation_min': float,
 '{const_name}_violation_mean': float,
 '{const_name}_violation_std': float}

In [153]:
from typing import Tuple
from dataclasses import dataclass, asdict


@dataclass
class ColumnDtype:
    run_id: object = "a"
    experiment_id: type = (object,)
    benchmark_id: type = (object,)
    energy: type = (object,)
    energy_min: type = (float,)
    energy_mean: type = (float,)
    energy_std: type = (float,)
    objective: type = (object,)
    obj_min: type = (float,)
    obj_mean: type = (float,)
    obj_std: type = (float,)
    num_feasible: type = (int,)
    num_samples: type = (int,)
    
    def __post_init__(self, const_name):
        self.const = const_name
        
        setattr(self, "const", const_name)
        violations: Tuple[Tuple[str, type]] = (
            ("{const_name}_violations", object),
            ("{const_name}_violation_min", float),
            ("{const_name}_violation_mean", float),
            ("{const_name}_violation_std", float),
        )
        
    def get_columns(self):
        self.keys()
    

c = ColumnDtype("aaa")
c.const


AttributeError: 'ColumnDtype' object has no attribute 'const'

In [3]:
import sys

sys.path.append("../") 

import jijbench as jb

experiment = jb.Experiment()

In [8]:
experiment._table.get_dtypes()

{'run_id': object,
 'experiment_id': object,
 'benchmark_id': object,
 'timestamp': pandas._libs.tslibs.timestamps.Timestamp,
 'energy': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objective': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_occurances': object,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object,
 '{const_name}_violation_min': float,
 '{const_name}_violation_mean': float,
 '{const_name}_violation_std': float}

In [213]:
json_str = "{\"a\": {\"b\": 1}}"
print(json_str)
print(json.loads(json_str))
print(type(json.loads(json_str)))

{'a': {"b": 1}}


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [221]:
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.inf)

df = pd.DataFrame([[1., 2., 3.], [2., 3., 4.]])
df.at[0, "list"] = object
df.at[0, "list"] = [[1., 2., 3.], [1., 2., 3.]]
df.at[0, "array"] = object
df.at[0, "array"] = np.random.normal(size=(10,5,4,3,2))
df.at[0, "dict"] = object
df.at[0, "dict"] = {"a": {"b": 1}}


df

,0,1,2,list,array,dict
0,1.0,2.0,3.0,"[[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]]","[[[0.6336372627527503, -0.8595895768799455], [...",{'a': {'b': 1}}
1,2.0,3.0,4.0,NaN,NaN,NaN


In [220]:
class A:
    def __init__(self):
        self.a = 1
        
    def method(self, x):
        print(self)
        print(x)

A.method("a", 1)

a
1


In [217]:
df.loc[0].to_dict()

{0: 1.0,
 1: 2.0,
 2: 3.0,
 'list': [[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]],
 'array': array([[[[[ 8.58198797e-01, -5.00281048e-01,  7.18558388e-02,
            -8.94875125e-01],
           [ 9.63282311e-01,  5.51307617e-01,  2.23055923e+00,
             1.00004810e+00],
           [ 8.08116760e-01,  1.55381314e+00,  1.13366374e+00,
             1.03907525e+00]],
 
          [[-7.11809129e-02,  1.64354089e+00,  7.89208702e-01,
             4.76333197e-01],
           [-1.16166625e+00, -3.16467435e-01, -1.39335784e+00,
             7.42164051e-01],
           [ 9.03981399e-01,  1.50897340e+00, -8.70694518e-01,
            -7.99657673e-01]],
 
          [[ 5.91733770e-01, -7.09624713e-01, -4.82635512e-01,
            -7.23546706e-01],
           [-1.53004656e-01, -2.43822177e+00, -5.13552334e-01,
             7.06947621e-01],
           [-1.08872935e+00,  3.09911189e-01, -1.11693901e+00,
             1.21510595e+00]],
 
          [[ 8.92699706e-01, -1.07052825e+00, -2.79946423e-01,
          

In [203]:
print(type(df.loc[0, "list"]))
print(type(df.loc[0, "array"]))
print(type(df.loc[0, "dict"]))

<class 'list'>
<class 'numpy.ndarray'>
<class 'dict'>


In [222]:
df.to_csv("test.csv")

In [223]:
df = pd.read_csv("test.csv", index_col=0)


In [224]:
df

,0,1,2,list,array,dict
0,1.0,2.0,3.0,"[[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]]",[[[ 0.63363726 -0.85958958]\n [ 1.75861345 -0...,{'a': {'b': 1}}
1,2.0,3.0,4.0,NaN,NaN,NaN


In [186]:
df.loc[0, "array"]

In [199]:
import re

def _replace_numpystr(x):
    try:
        return re.sub(
            r"\n+",
            "",
            re.sub(
                r" +",
                ", ",
                re.sub(
                    r"\[ +",
                    "[",
                    re.sub(
                        r" +\]", "]", re.sub(r"\.\]", ".0]", re.sub(r"\. ", ".0 ", x))
                    ),
                ),
            ),
        )
    except TypeError:
        return x

df = pd.read_csv("test.csv", index_col=0)
#df["array"] = df["array"].map(_replace)
#df

# regex = re.compile(r"\n+ ")
s = df.loc[0, "array"]
# re.findall(regex, s)

s = _replace_numpystr(s)
# regex = re.compile(r"\.[ \]]")
# s = re.sub(regex, ".")

series = df["array"].map(_replace_numpystr)
series = series.map(_loads)
series = series.map(np.array)


numpy.ndarray

In [189]:
np.array(json.loads(s)).shape

(10, 2, 5, 3, 4)

In [67]:
import json

def _loads(_x):
    try:
        return json.loads(_x)
    except (TypeError, json.JSONDecodeError):
        return _x
    
df = pd.read_csv("test.csv", index_col=0)
#print(df.loc[0, "list"])
#print(type(df.loc[0, "list"]))
#df["list"] = df["list"].map(_loads)
#print(type(df.loc[0, "list"]))

print()
print(df.loc[0, "array"])
s = df.loc[0, "array"].replace(". ", ".0 ").replace(".]", ".0]").replace("\n ", "*").replace(" ", ", ").replace("*", ", ")

print(s)
array = np.array(json.loads(s))





[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
[[[0.0, 0.0], , [0.0, 0.0]]
, [[0.0, 0.0], , [0.0, 0.0]]]


JSONDecodeError: Expecting value: line 1 column 15 (char 14)

In [201]:
json.loads("a")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [49]:
print(type(df.loc[0, "list"]))
print(type(df.loc[0, "array"]))
print(type(df.loc[0, "dict"]))

<class 'str'>
<class 'str'>
<class 'str'>


In [92]:
arraystr = df.loc[0, "list"]
arraystr = str([[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]])
array = np.fromstring(arraystr, sep=",")
array

/tmp/ipykernel_10963/683216600.py:3: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  array = np.fromstring(arraystr, sep=",")


array([], dtype=float64)

In [97]:
import re
s = df.loc[0, "list"]
# s = "[1, 2 ,3]"
s = arraystr
s = re.split("[\[\]]", s)
dim = 0
for c in s:
    if not c:
        dim += 1
    else:
        break
    
obj = []
for c in s:
    obj.append()



TypeError: list.append() takes exactly one argument (0 given)

In [100]:
s


['',
 '',
 '',
 '1, 2, 3',
 ', ',
 '1, 2, 3',
 '',
 ', ',
 '',
 '1, 2, 3',
 ', ',
 '1, 2, 3',
 '',
 '',
 '']

In [103]:
s = df.loc[0, "list"]
# s = "[1, 2 ,3]"
# s = arraystr
s = re.split("[\[\]]", s)
s

['', '', '1, 2, 3', ', ', '1, 2, 3', '', '']

In [12]:
import json

s = str([[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]])
print(type(json.loads(s)))

x = np.zeros((2, 2))
json.loads(str(x.tolist()))

<class 'list'>


[[0.0, 0.0], [0.0, 0.0]]

In [4]:
s

NameError: name 's' is not defined

In [140]:
[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]]

[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]]

In [18]:

try:
    json.loads("str")
except (TypeError, json.JSONDecodeError):
    pass



TypeError: the JSON object must be str, bytes or bytearray, not float

In [19]:
json.loads("str")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [106]:
[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]]]

[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]],
 [[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]]]

In [86]:
s

['', '', '1, 2, 3', ', ', '1, 2, 3', '', '']

In [23]:
print(type(df.loc[0, 3]))
print(type(df.loc[0, 4]))

<class 'list'>
<class 'dict'>


In [24]:
import numpy as np

a = np.zeros(2)

type(a)

numpy.ndarray

In [25]:
df.at[3, 1] = np.zeros()

,0,1,2,3,4
0,1,2,3,"[1, 2, 3]",{'a': 1}
1,2,3,4,NaN,NaN


In [94]:
import jijbench as jb


class Evaluator:
    def __init__(self, experiment: jb.Experiment):
        self.experiment = experiment
    
    @property
    def table(self):
        return self.experiment.table
    
    @property
    def artifact(self):
        return self.experiment.artifact
    
    def aggregate(self):
        pass
    
    def time_to_solution(self, criteria, pr=0.99, expand=True, column="TTS"):
        scorer = make_scorer(_time_to_solution, criteria=criteria, pr=pr)
        tts = self.table.apply(scorer, axis=1)
        if expand:
            self.table[column] = tts
        return tts
    
    def time_to_feasible_solution(self):
        NotImplementedError
        
    def time_to_most_derived_solution(self):
        NotImplementedError
        
    
    def success_probability(self, criteria, expand=True, column="success_probability"):
        scorer = make_scorer(_success_probability, criteria=criteria)
        ps = self.table.apply(scorer, axis=1)
        if expand:
            self.table[column] = ps
        return 
    
    def feasible_rate(self):
        NotImplementedError
        
    def residual_energy(self):
        NotImplementedError

class _Scorer:
    def __init__(self, score_func, kwargs):
        self._score_func = score_func
        self._kwargs = kwargs
        
    def __call__(self, x):
        return self._score_func(x, **self._kwargs)


def make_scorer(score_func, **kwargs):
    return _Scorer(score_func, kwargs)


def _time_to_solution(x, criteria, pr):
    ps = _success_probability(x, criteria) + 1e-16
    return np.log(1 - pr) / np.log(1 - ps) * x.execution_time if ps < pr else x.execution_time


def _success_probability(x, criteria):
    return (x.energy <= criteria).sum() / len(x.energy)

In [52]:
import numpy as np
scorer = _Scorer(_success_probability, criteria=0)
scorer(np.array([0, 0, 0, 0, 0]))

{'criteria': 0}


1.0

In [2]:
import openjij as oj

sampler = oj.SASampler(num_reads=10)
experiment = jb.Experiment(autosave=False)
for _ in range(3):
    with experiment.start():
        response = sampler.sample_qubo({(0, 1): 1})
        experiment.store({"result": response})
experiment.table.dropna(axis="columns", inplace=True)

In [54]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,num_occurances,sampling_time,execution_time
0,0d2b4aa6-6d68-4e97-8e51-e2a87d80a8b7,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.123263,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1717.828214,152.932480
1,2fe951d8-957b-4364-bd58-3ccf77a95482,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.131373,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1661.228016,150.022656
2,e45f999e-a5e2-424d-a351-41bc312d9ad6,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.138370,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1724.627800,152.162462


In [23]:
experiment.table.energy.map(lambda x: (x <= 0).sum())

0    10
1    10
2    10
Name: energy, dtype: int64

In [11]:
len(experiment.table.energy[0])

10

In [4]:
response.energies

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [95]:
evaluator = Evaluator(experiment)
evaluator.success_probability(10)

In [96]:
evaluator.tts(0)

0    152.932480
1    150.022656
2    152.162462
dtype: float64

In [97]:
evaluator.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,num_occurances,sampling_time,execution_time,success_probability,tts
0,0d2b4aa6-6d68-4e97-8e51-e2a87d80a8b7,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.123263,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1717.828214,152.932480,1.0,152.932480
1,2fe951d8-957b-4364-bd58-3ccf77a95482,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.131373,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1661.228016,150.022656,1.0,150.022656
2,e45f999e-a5e2-424d-a351-41bc312d9ad6,b487eaa1-dffb-45fe-994d-636a03b6d41a,db7ede88-29a8-43a7-8b72-c206237c20bb,2022-03-14 05:33:26.138370,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1724.627800,152.162462,1.0,152.162462
